In [1]:
import numpy as np
import pandas as pd
import datetime as dt
import bz2
import json
from tld import get_tld
import nltk
from nltk.stem import PorterStemmer, WordNetLemmatizer
from nltk.corpus import stopwords, wordnet
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel
import string
import math
from collections import Counter
from operator import itemgetter
import re
import pickle

nltk.download()
nltk.download('stopwords')
nltk.download('punkt')

showing info https://raw.githubusercontent.com/nltk/nltk_data/gh-pages/index.xml


[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/selene/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /Users/selene/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [2]:
from Data_clean_functions import *

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/selene/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /Users/selene/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


### CREATE PATHS: change according to local file

In [3]:
DATA_PATH = './data/'
FILE2019 = DATA_PATH + 'quotes-2019.json.bz2'
PATH_OUT = DATA_PATH + 'clean-quotes-2019.json.bz2'

In [4]:
df_base = pd.read_json(FILE2019, lines=True, compression='bz2', nrows=100)
df_base

,quoteID,quotation,speaker,qids,date,numOccurrences,probas,urls,phase
0,2019-10-14-000009,% 9: D: D @? 6 H2J H6 E9@F89E H6 4@F=5 8: G6 3...,None,[],2019-10-14 07:30:00,1,"[[None, 0.5595], [Julio Morales, 0.4405]]",[http://www.ivpressonline.com/news/local/ivc-f...,E
1,2019-04-08-048753,It is immoral. It is harmful. It is hurtful.,President Donald Trump,[Q22686],2019-04-08 16:22:00,44,"[[President Donald Trump, 0.5802], [None, 0.36...",[https://www.mercedsunstar.com/news/business/a...,E
2,2019-07-17-000030,"[ Amber ] loves her son more than anything,",None,[],2019-07-17 22:54:35,1,"[[None, 0.8276], [Amber Portwood, 0.1724]]",[https://www.inquisitr.com/5535969/did-amber-p...,E
3,2019-05-15-053302,It is important for our equine science student...,Sally Johnson,[Q42336656],2019-05-15 18:03:22,1,"[[Sally Johnson, 0.5721], [None, 0.4279]]",[https://www.lanereport.com/113381/2019/05/qua...,E
4,2019-04-20-000011,... an Afrikaans family living in South Africa...,None,[],2019-04-20 22:30:57,1,"[[None, 0.8331], [you long, 0.1669]]",[http://filmthreat.com/reviews/the-harvester/],E
...,...,...,...,...,...,...,...,...,...
95,2019-10-22-049545,"It is very unstructured, uncourt-like and it i...",Ellen Mirojnick,[Q15919390],2019-10-22 19:00:04,1,"[[Ellen Mirojnick, 0.7907], [None, 0.2093]]",[http://www.blastr.com/syfywire/ellen-mirojnic...,E
96,2019-09-26-000599,12 Movements from Romeo and Juliet,None,[],2019-09-26 00:00:00,2,"[[None, 0.5111], [Sergei Babayan, 0.2205], [Da...",[http://news.cornell.edu/stories/2019/09/thing...,E
97,2019-06-17-046011,It is vital that investigators get access to T...,Danielle Smith,"[Q22957142, Q5219439]",2019-06-17 00:00:00,21,"[[Danielle Smith, 0.5516], [None, 0.4484]]",[http://www.hitz939.com.au/news/national-news/...,E
98,2019-05-30-000684,20 new songs that will change your lives.,Liam Gallagher,"[Q216708, Q30084404]",2019-05-30 11:27:45,3,"[[Liam Gallagher, 0.8319], [None, 0.1503], [Gr...",[https://www.nme.com/news/music/liam-gallagher...,E


## Create a dictionnary of categories and associated synonyms

In [5]:
matchers = {"art": ["art", "paint", "draw", "museum"], \
            "business": ["business", "finance", "economy", "commerce", "bank", "money", "trade"], \
            "entertainment":["entertainment"], 
            "fashion":["fashion", "couture", "designer"], \
            "medicine":["medicine", "health", "pharmacy", "wellbeing", "body"], \
            "music":["music", "song", "album", "concert"], \
            "politics":["politics", "government"], \
            "science":["science", "research"], \
            "sport": ["sport", "football", "athletics", "swimming", "rugby", "tennis", "volleyball"]}

In [ ]:
# Don't run this as doesn't return good list

'''

def find_synonyms(matchers):
    for category in matchers:
        synonyms = []
        for i in range(len(matchers[category])):
            word = matchers[category][i]
            for syn in wordnet.synsets(word):
                for l in syn.lemmas():
                    synonyms.append(l.name())
            matchers[category] = synonyms
    return matchers

matchers = find_synonyms(matchers)

'''

In [6]:
generalizeDictionary(matchers)

In [7]:
matchers

{'art': ['art', 'paint', 'draw', 'museum'],
 'business': ['busi',
  'financ',
  'economi',
  'commerc',
  'bank',
  'money',
  'trade'],
 'entertainment': ['entertain'],
 'fashion': ['fashion', 'coutur', 'design'],
 'medicine': ['medicin', 'health', 'pharmaci', 'wellb', 'bodi'],
 'music': ['music', 'song', 'album', 'concert'],
 'politics': ['polit', 'govern'],
 'science': ['scienc', 'research'],
 'sport': ['sport',
  'footbal',
  'athlet',
  'swim',
  'rugbi',
  'tenni',
  'volleybal']}

### PROCESSING DATA

In [20]:
# With pickle

with pd.read_json(FILE2019, lines=True, compression='bz2', chunksize=1000) as df_reader:
        with bz2.open(PATH_OUT, 'wb') as d_file:
            for chunk in df_reader:
                chunk_cleaned, chunk_length = process_chunk_complete(chunk, 0.5, matchers)
                pickle.dump(chunk_cleaned, d_file)

Processing chunk with 1000 rows
Processing chunk with 1000 rows
Processing chunk with 1000 rows
Processing chunk with 1000 rows
Processing chunk with 1000 rows
Processing chunk with 1000 rows


KeyboardInterrupt: 

In [25]:
filehandler = bz2.open(PATH_OUT, 'rb') 
dataframe_example = pickle.load(filehandler)
dataframe_example

,quoteID,quotation,speaker,qids,date,numOccurrences,probas,urls,phase,sitenames,domain,tags
0,2019-04-08-048753,It is immoral. It is harmful. It is hurtful.,President Donald Trump,[Q22686],2019-04-08 16:22:00,44,"[[President Donald Trump, 0.5802], [None, 0.36...",[https://www.mercedsunstar.com/news/business/a...,E,"[mercedsunstar, sacbee, mynorthwest, lasvegass...","[com, com, com, com, com, com, com, com, com, ...","[[business], [business], [], [], [politics], [..."
1,2019-05-15-053302,It is important for our equine science student...,Sally Johnson,[Q42336656],2019-05-15 18:03:22,1,"[[Sally Johnson, 0.5721], [None, 0.4279]]",[https://www.lanereport.com/113381/2019/05/qua...,E,[lanereport],[com],[[]]
2,2019-02-27-055406,It is important to many Native American tribes...,Rafael Ortega,"[Q16672061, Q3417253, Q3417255, Q48410107]",2019-02-27 00:00:00,1,"[[Rafael Ortega, 0.7587], [None, 0.2413]]",[http://kstp.com/news/riverview-corridor-proje...,E,[kstp],[com],[[]]
3,2019-12-08-023053,"It is impossible, biologically, truly to `rest...",Barry Lopez,[Q809063],2019-12-08 06:00:00,1,"[[Barry Lopez, 0.8142], [None, 0.1858]]",[https://www.timescolonist.com/opinion/op-ed/i...,E,[timescolonist],[com],[[]]
4,2019-02-21-000088,[ Chilton ] put it on a little tape recorder a...,Sam the Sham,[Q1971786],2019-02-21 11:05:34,1,"[[Sam the Sham, 0.6472], [None, 0.3278], [Alex...",[http://www.nashvillescene.com/music/features/...,E,[nashvillescene],[com],[[music]]
...,...,...,...,...,...,...,...,...,...,...,...,...
608,2019-12-28-019903,It's kind of fun to see it. Just to go over th...,Maggie Thompson,[Q733534],2019-12-28 14:00:00,1,"[[Maggie Thompson, 0.7961], [None, 0.2039]]",[http://www.expressnews.com/news/local/article...,E,[expressnews],[com],[[]]
609,2019-04-24-003931,"And with a book called Death Threat, I just fe...",Vivek Shraya,[Q7937601],2019-04-24 20:41:39,1,"[[Vivek Shraya, 0.8852], [None, 0.1148]]",[https://www.straight.com/arts/1232356/verses-...,E,[straight],[com],[[art]]
610,2019-10-21-048385,It's less about what they say to each other an...,Nathan Felix,[Q22277761],2019-10-21 00:00:00,1,"[[Nathan Felix, 0.699], [None, 0.1455], [Domin...",[https://www.tpr.org/post/mcnay-hopes-become-n...,E,[tpr],[org],"[[art, art]]"
611,2019-02-02-003282,"And, you allow the challenge flag to be used o...",Reggie Bush,[Q555271],2019-02-02 20:07:21,7,"[[Reggie Bush, 0.9156], [None, 0.0827], [Cardi...",[http://kron4.com/sports/the-big-game/how-regg...,E,"[kron4, krqe, wwlp, everythinglubbock, illinoi...","[com, com, com, com, net, com, com]","[[sport], [sport], [sport], [sport], [sport], ..."


### Without pickle / writing to new json

In [ ]:
with pd.read_json(FILE2016, lines=True, compression='bz2', chunksize=1000) as df_reader:
    with bz2.open(PATH_OUT, 'wb') as d_file:
        for chunk in df_reader:
            chunk_cleaned, chunk_length = process_chunk_complete(chunk, 0.5, matchers)
            chunk_json = chunk_cleaned.to_json(orient='columns')#, index=False) #write it to pickle file instead
            d_file.write((chunk_json+'\n').encode('utf-8'))

In [ ]:
df = pd.read_json(DATA_PATH + 'clean-quotes-2019.json.bz2', orient='columns', compression='bz2')
df

In [ ]:
df = pd.read_json(DATA_PATH + 'clean-quotes-2019.json.bz2', orient='columns', compression='bz2', lines=True, nrows=100)
df

In [ ]:
with pd.read_json(DATA_PATH + 'clean-quotes-2019.json.bz2', lines=True, compression='bz2', chunksize=1000) as df_reader:
    